In [33]:
import pandas as pd
import psycopg2
import yfinance as yf
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")

load_dotenv()

True

In [34]:
import os
from sqlalchemy import create_engine, text

host = os.environ["SUPABASE_HOST"]
port = os.environ.get("SUPABASE_PORT", 5432)
db   = os.environ["SUPABASE_DB"]
user = os.environ["SUPABASE_USER"]
pw   = os.environ["SUPABASE_PASSWORD"]

url = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}?sslmode=require"
engine = create_engine(url, connect_args={"sslmode":"require"}, pool_pre_ping=True)

# 연결 테스트
with engine.begin() as conn:
    print(conn.execute(text("select now()")).scalar())


2025-08-12 13:28:10.792131+00:00


In [38]:
tickers = ["AAPL", "MSFT", "TSLA"]
period = "10y"
interval = "1d"

df = yf.download(tickers, period=period, interval=interval, group_by="ticker", auto_adjust=True)
df = df.stack(level=0)
df = df.reset_index()
df.columns = df.columns.str.lower()

[*********************100%***********************]  3 of 3 completed


In [39]:
with engine.begin() as conn:
    df.to_sql("stock_data", con=conn, if_exists="replace", index=False)

In [40]:
with engine.begin() as conn:
    df_test = pd.read_sql("SELECT * FROM stock_data", engine)

In [41]:
df_test.head()

,date,ticker,open,high,low,close,volume
0,2015-08-12,AAPL,25.288369,25.937826,24.636664,25.897375,404870000
1,2015-08-12,MSFT,40.050918,40.666555,39.634715,40.527821,30181400
2,2015-08-12,TSLA,15.666667,15.984667,15.516000,15.878000,55920000
3,2015-08-13,AAPL,26.077151,26.158053,25.740063,25.877146,194143200
4,2015-08-13,MSFT,40.805295,40.839976,40.311054,40.519154,22627200
